# LSTM vs RNN: Handling Long-Term Dependencies
This notebook demonstrates the problem of long-term dependencies with a vanilla RNN and how an LSTM solves it using a synthetic dataset.

In [ ]:
!pip install numpy matplotlib tensorflow scikit-learn

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

## Generate Synthetic Data

In [ ]:
def generate_data(sequence_length=50, size=1000):
    np.random.seed(42)
    X = np.linspace(0, 100, size)
    Y = np.sin(X)  # Long-term dependency: sine wave
    # Add noise to simulate a real-world problem
    Y += np.random.normal(0, 0.1, size)
    data = []
    labels = []
    for i in range(len(Y) - sequence_length):
        data.append(Y[i:i + sequence_length])
        labels.append(Y[i + sequence_length])
    return np.array(data), np.array(labels)

sequence_length = 50
X, y = generate_data(sequence_length)
X = X.reshape((X.shape[0], X.shape[1], 1))  # Add a feature dimension
scaler = MinMaxScaler(feature_range=(0, 1))
X = scaler.fit_transform(X.reshape(-1, 1)).reshape(X.shape)
y = scaler.fit_transform(y.reshape(-1, 1))

# Split into training and testing sets
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

## Define and Train RNN Model

In [ ]:
rnn_model = Sequential([
    SimpleRNN(10, activation='tanh', input_shape=(sequence_length, 1)),
    Dense(1)
])
rnn_model.compile(optimizer='adam', loss='mse')
rnn_history = rnn_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test), verbose=0)

## Define and Train LSTM Model

In [ ]:
lstm_model = Sequential([
    LSTM(10, activation='tanh', input_shape=(sequence_length, 1)),
    Dense(1)
])
lstm_model.compile(optimizer='adam', loss='mse')
lstm_history = lstm_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test), verbose=0)

## Evaluate Both Models

In [ ]:
rnn_predictions = rnn_model.predict(X_test)
lstm_predictions = lstm_model.predict(X_test)

# Inverse transform predictions and true values
y_test = scaler.inverse_transform(y_test)
rnn_predictions = scaler.inverse_transform(rnn_predictions)
lstm_predictions = scaler.inverse_transform(lstm_predictions)

# Calculate RMSE for both models
rnn_rmse = np.sqrt(mean_squared_error(y_test, rnn_predictions))
lstm_rmse = np.sqrt(mean_squared_error(y_test, lstm_predictions))

print(f"RNN RMSE: {rnn_rmse}")
print(f"LSTM RMSE: {lstm_rmse}")

## Visualize Predictions

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(y_test, label='True Values')
plt.plot(rnn_predictions, label='RNN Predictions')
plt.plot(lstm_predictions, label='LSTM Predictions')
plt.legend()
plt.title("True Values vs RNN and LSTM Predictions")
plt.show()